https://github.com/quz2004/COS243/blob/main/RAG_llamaIndex_tutorial.md

In [2]:
%pip install -q llama-index==0.10.18 llama-index-llms-groq==0.1.3 groq==0.4.2 llama-index-embeddings-huggingface==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00


In [3]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
# import os
# from dotenv import load_dotenv
# load_dotenv()
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
from google.colab import userdata
API_KEY = userdata.get('GROQ_API_KEY')

In [5]:
!wget https://myweb.sabanciuniv.edu/rdehkharghani/files/2016/02/The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf

--2024-11-15 19:52:30--  https://myweb.sabanciuniv.edu/rdehkharghani/files/2016/02/The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf
Resolving myweb.sabanciuniv.edu (myweb.sabanciuniv.edu)... 159.20.64.81
Connecting to myweb.sabanciuniv.edu (myweb.sabanciuniv.edu)|159.20.64.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12585336 (12M) [application/pdf]
Saving to: ‘The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf’

The-Morgan-Kaufmann 100%[===================>]  12.00M  5.18MB/s    in 2.3s    

2024-11-15 19:52:33 (5.18 MB/s) - ‘The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf’ saved [12585336/12585336]



In [6]:
# data ingestion
reader = SimpleDirectoryReader(input_files=["./The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf"])
documents = reader.load_data()

In [7]:
len(documents)

740

In [8]:
documents[30].metadata

{'page_label': 'xxx',
 'file_name': 'The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf',
 'file_path': 'The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf',
 'file_type': 'application/pdf',
 'file_size': 12585336,
 'creation_date': '2024-11-15',
 'last_modified_date': '2016-02-10'}

In [9]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/740 [00:00<?, ?it/s]

In [10]:
len(nodes)

784

In [11]:
nodes[0].metadata

{'page_label': 'Cover',
 'file_name': 'The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf',
 'file_path': 'The-Morgan-Kaufmann-Series-in-Data-Management-Systems-Jiawei-Han-Micheline-Kamber-Jian-Pei-Data-Mining.-Concepts-and-Techniques-3rd-Edition-Morgan-Kaufmann-2011.pdf',
 'file_type': 'application/pdf',
 'file_size': 12585336,
 'creation_date': '2024-11-15',
 'last_modified_date': '2016-02-10'}

In [12]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
import numpy as np

from tabulate import tabulate

# Initialize the embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Sample texts
texts = [
    "The weather is nice today.",
    "I love eating pizza for dinner.",
    "Apple just released iPhone 17",
    "This red apple smells good",
    "Tesla stock rises as Trump wins the election",
    "Should I Sell My Individual Stocks and Reinvest in S&P 500/Nasdaq 100?"
]

# Generate embeddings for the texts
embeddings = [embed_model.get_text_embedding(text) for text in texts]

# Function to calculate cosine similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Calculate similarities and prepare table data
table_data = []
for i in range(len(texts)):
    row = [f"Text {i+1}"]
    for j in range(len(texts)):
        if i == j:
            row.append("1.0000")
        elif j > i:
            similarity = cosine_similarity(embeddings[i], embeddings[j])
            row.append(f"{similarity:.4f}")
        else:
            row.append("")
    table_data.append(row)

# Prepare headers
headers = [""] + [f"Text {i+1}" for i in range(len(texts))]

# Print the table
print(tabulate(table_data, headers=headers, tablefmt="grid"))

# Print the text content for reference
print("\nText content:")
for i, text in enumerate(texts, 1):
    print(f"Text {i}: {text}")

+--------+----------+----------+----------+----------+----------+----------+
|        | Text 1   | Text 2   | Text 3   | Text 4   | Text 5   |   Text 6 |
+========+==========+==========+==========+==========+==========+==========+
| Text 1 | 1.0000   | 0.1364   | 0.0529   | 0.1190   | 0.0408   |   0.0057 |
+--------+----------+----------+----------+----------+----------+----------+
| Text 2 |          | 1.0000   | -0.0100  | 0.0830   | 0.0277   |   0.0185 |
+--------+----------+----------+----------+----------+----------+----------+
| Text 3 |          |          | 1.0000   | 0.2874   | 0.1171   |   0.0379 |
+--------+----------+----------+----------+----------+----------+----------+
| Text 4 |          |          |          | 1.0000   | 0.1528   |  -0.0115 |
+--------+----------+----------+----------+----------+----------+----------+
| Text 5 |          |          |          |          | 1.0000   |   0.2119 |
+--------+----------+----------+----------+----------+----------+----------+

In [14]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [15]:
llm = Groq(model="llama-3.1-70b-versatile", api_key=API_KEY)

In [16]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

In [ ]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Parsing nodes:   0%|          | 0/740 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/784 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir /content/drive/MyDrive/Gen.AI/DB

In [ ]:
vectDB = "/content/drive/MyDrive/Gen.AI/DB/textbookstorage"

In [ ]:
vector_index.storage_context.persist(persist_dir=vectDB)

In [ ]:
storage_context = StorageContext.from_defaults(persist_dir=vectDB)

In [ ]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine(service_context=service_context)

In [ ]:
query = "Explain cosine similarity"
resp = query_engine.query(query)

In [ ]:
import textwrap

def Print(text, width=80, **args):
    lines = text.split('\n')  # Split the text into lines based on original line breaks
    wrapped_lines = []
    for line in lines:
        wrapped_lines.extend(textwrap.wrap(line, width=width))  # Wrap each line individually
    print('\n'.join(wrapped_lines), **args)


In [ ]:
Print(resp.response)

In [ ]:
for node in resp.source_nodes:
    # Access the TextNode object directly
    text_node = node.node

    # Assuming metadata is stored within the TextNode's metadata
    source = text_node.metadata.get('file_name') # Access metadata using .metadata.get()
    page = text_node.metadata.get('page_label')  # Access metadata using .metadata.get()

    Print(f"Source: {source}")
    Print(f"Page: {page}")

In [ ]:
query = """Create concept check questions and answer about cosine similarity.
Format output in markdown with 2 line breaks to separate markdown entities"""
resp = query_engine.query(query)
Print(resp.response)

QUIZ